In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import vigra
import numpy as np
import os
import argparse
from itertools import repeat


from functools import partial

from segmfriends.utils.various import parse_data_slice
from segmfriends.utils import various as sgm_utils
import time
import h5py

import neuroglancer

# if 'viewer' not in globals():
neuroglancer.set_server_bind_address(bind_port=7010)
viewer = neuroglancer.Viewer()

print(viewer)

from PIL import Image
from segmfriends.utils.various import writeHDF5, readHDF5, get_hdf5_inner_paths


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
http://127.0.0.1:7010/v/5bd72cb55f1f12c70df3001df420dd4e094dc35e/


In [17]:
main_data_dir = "/scratch/bailoni/projects/train_cellpose/hdf5_data"

datasets = [
        "alex",
        "LIVECell_test"
]

model_names = {
    "cyto2_diamEst": "CellPose cyto2 model",
                      # "cyto_diamEst": "CellPose cyto",
                      "trained_on_LIVECell_noDiamEst": "Trained on LIVECell+CellPose from scratch",
                      # "trained_on_LIVECell_diamEst": "Trained on LIVECell+CellPose data (est diam)",
                      # "trained_on_cellpose_noDiamEst": "Trained on CellPose data",
                      # "trained_on_cellpose_diamEst": "Trained on CellPose data (est diam)",
                      # "finetuned_LIVECell_lr_02_noDiamEst": "cyto2 finetuned on LIVECell+CellPose",
                      "finetuned_LIVECell_lr_00002_noDiamEst": "cyto2 finetuned on LIVECell+CellPose",
                      # "finetuned_LIVECell_lr_00002_diamEst": "finetuned_LIVECell_lr_00002_diamEst",
                      # "finetuned_LIVECell_lr_02_diamEst": "finetuned_LIVECell_lr_02_diamEst",
}

# Get data:
GT = readHDF5(os.path.join(main_data_dir, "LIVECell_train", "raw_and_gt.h5"), "gt")
raw = readHDF5(os.path.join(main_data_dir, "LIVECell_train", "raw_and_gt.h5"), "raw")
# segm = readHDF5(os.path.join(main_data_dir, "LIVECell_train", "cyto2_diamEst.h5"), "data")

### Visualize

In [18]:
dimensions=neuroglancer.CoordinateSpace(
                scales=[1, 1, 1],
                units=['nm', 'nm', 'nm'],
                names=['x', 'y', 'z'])


neuroglancer.AnnotationLayer()
with viewer.txn() as s:
    s.layers['raw'] = neuroglancer.ImageLayer(
      source=neuroglancer.LocalVolume(
          raw,
          dimensions
      ))
    s.layers['GT'] = neuroglancer.SegmentationLayer(
      source=neuroglancer.LocalVolume(
          GT.astype('uint32'),
          dimensions
      ))
#     s.layers['segm'] = neuroglancer.SegmentationLayer(
#       source=neuroglancer.LocalVolume(
#           segm.astype('uint32'),
#           dimensions
#       ))
    s.layers['annotations'] = neuroglancer.AnnotationLayer()

